<font color = '#4682B4'>

## The Evolution of Portuguese Public Contracts Value across Time, Location and Service Type ##
</font>

#### Group C3:
João Bilé (20241345)
Carolina Carvalho (20241349)
Gabriel Domenech (20241354)
Md Haque (20241356)

# 📚 Table of Contents

1. **Introduction**

2. **Possible analysis**

3. **Main data sources**

4. **Imports**

5. **Data exploration**

   a. Inspecting shapes

   b. Inspecting data types

   c. Inspecting null values

   d. Standardized Column Mapping Across Datasets

   e. Data exploration conclusions

7. **Preprocessing data**


## Introduction

The project aims to provide an overview of how public spending evolved from multiple perspectives. We are going to analyze contracts by sector, value, inflation, and other perspectives to undertand this scenario.

## Project scope
This project aims to investigate how the value of public contracts in Portugal has evolved over time, with a particular focus on the impact of inflation and major external events. The following questions guide our exploration:

- Regional differences: Have all regions of the country been equally affected by price changes and contract volume shifts?

- Inflation-adjusted growth: Has the average value of public contracts kept pace with inflation, or have certain types of contracts grown disproportionately?

- Sectoral trends: Which sectors (e.g., education, construction, healthcare) have experienced the largest increases in contract value over time?

- Contractor dynamics: Are there specific companies that have seen a significant rise in the volume or value of contracts awarded?

- PRR influence: What is the role of the Recovery and Resilience Plan (PRR) in shaping recent public contracting patterns, and which sectors have been most influenced by it?

- Impact of global crises: Did extreme events such as the COVID-19 pandemic or the war in Ukraine affect the scale and type of public procurement?

## Main data sources

The primary datasets consists of public contract records published by the Portuguese government (https://www.base.gov.pt/ and https://www.dados.gov.pt), covering the period from 2018 to 2025, including information related to PRR – Recovery and Resilience Plan contracts.

- Additional Data Sources
To enrich the analysis and contextualize trends, additional public datasets may be incorporated, such as:

    - INE (Instituto Nacional de Estatística): Economic indicators, inflation rates, sector-level statistics.

    - PORDATA: Demographic, regional, and industry-level data to support regional and sectoral comparisons.

    - Dados.gov.pt: Public procurement records, entity-level information, and PRR contract metadata.

These sources help link contract dynamics to macroeconomic variables and provide deeper insight into public spending patterns.

## Imports

#### **Importing needed libraries**

| **Library**                        | **Description**                                                                                                                                                   |
|----------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Pandas (`pd`)**                | Data manipulation and analysis, providing easy-to-use data structures and tools for handling large datasets.                                                     |
| **NumPy (`np`)**                 | Supports multi-dimensional arrays and matrices, with mathematical functions for operating on these arrays.                                                       |
| **Matplotlib.pyplot (`plt`)**    | Visualization library with an interface for creating 2D plots (lines, bars, scatter, etc.).                                                                      |
| **Matplotlib.ticker (`mticker`)**| Tools for controlling tick placement and formatting in Matplotlib plots.                                                                                         |
| **Seaborn (`sns`)**              | Built on top of Matplotlib, it simplifies creating attractive and informative statistical visualizations.                                                       |
| **AST (`ast`)**                  | Parses Python expressions into Abstract Syntax Trees — useful for safely evaluating strings that represent Python literals (e.g., list-like strings).           |
| **Datetime (`datetime`)**        | Provides classes for manipulating dates and times in both simple and complex ways.                                                                               |
| **OS (`os`)**                    | Provides functions to interact with the operating system, such as file and directory operations.                                                                 |
| **RE (`re`)**                    | Supports regular expressions for advanced string matching, searching, and manipulation.                                                                          |
| **IPython.display (`display`)**  | Enables rich output display in Jupyter Notebooks (e.g., pretty-printing dataframes, displaying HTML, images, etc.).                                              |


In [215]:
# Importing libraries detailed above for data analysis and machine learning

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import matplotlib.ticker as mticker
from datetime import datetime
sns.set(style="whitegrid")
import os
import re
from IPython.display import display


#### **Importing data**

In [217]:
base_path = 'Group_C3'

In [218]:
# Load datasets
csv_paths = [
    "contracts_2018.csv",
    "contracts_2019.csv",
    "contracts_2020.csv",
    "contracts_2021.csv",
    "contracts_2022.csv",
    "CPV_Macros.csv"
]
json_paths = [
    'Contratos2023.json',
    'Contratos2024.json',
    'Contratos2025.json'
]
excel_paths  = [
    'listagem-de-contratos-do-prr-20250519.xlsx'
]

In [219]:
# Load all dataset individually
csv_dfs = []
for filename in csv_paths:
    full_path = os.path.join(os.path.dirname(base_path), filename)
    if os.path.exists(full_path):
        df = pd.read_csv(full_path, encoding='utf-8', low_memory=False)
        csv_dfs.append(df)
    else:
        print(f"CSV file not found: {full_path}")

In [220]:
json_dfs = []
for filename in json_paths:
    full_path = os.path.join(os.path.dirname(base_path), filename)
    if os.path.exists(full_path):
        df = pd.read_json(full_path, encoding='utf-8')
        json_dfs.append(df)
    else:
        print(f"JSON file not found: {full_path}")

In [221]:
excel_dfs = []
for filename in excel_paths:
    full_path = os.path.join(os.path.dirname(base_path), filename)
    if os.path.exists(full_path):
        df = pd.read_excel(full_path)
        excel_dfs.append(df)
    else:
        print(f"Excel file not found: {full_path}")

## Data exploration

#### **Inspecting shapes**

In [224]:
# Create a mapping of dataset names to their DataFrames
datasets = {
    'df_2018': csv_dfs[0],
    'df_2019': csv_dfs[1],
    'df_2020': csv_dfs[2],
    'df_2021': csv_dfs[3],
    'df_2022': csv_dfs[4],
    'df_cpv': csv_dfs[5],
    'df_2023': json_dfs[0],
    'df_2024': json_dfs[1],
    'df_2025': json_dfs[2],
    'df_prr':  excel_dfs[0]
}

# Print each dataframe's shape
for name, df in datasets.items():
    print(f"{name}: {df.shape}")


df_2018: (204637, 27)
df_2019: (286866, 27)
df_2020: (342251, 27)
df_2021: (448597, 27)
df_2022: (444085, 27)
df_cpv: (75, 2)
df_2023: (191496, 35)
df_2024: (218972, 35)
df_2025: (75234, 35)
df_prr: (4355, 10)


#### **Inspecting data types**


In [226]:
# Inspecting the data types of each column, the number of non-null entries, and overall memory usage.
# Useful for understanding the dataset structure and identifying potential data quality issues.

for name, df in datasets.items():
    print(f"{name}: {df.info()}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204637 entries, 0 to 204636
Data columns (total 27 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Unnamed: 0               204637 non-null  int64  
 1   contract_id              204637 non-null  int64  
 2   name_contestants         118716 non-null  object 
 3   nif_contestants          204637 non-null  object 
 4   name_contracted          204636 non-null  object 
 5   nif_contracted           204637 non-null  object 
 6   contract_type            204637 non-null  object 
 7   invitees                 44 non-null      object 
 8   centralized_procedure    204631 non-null  object 
 9   procedure_type           204637 non-null  object 
 10  name_contracting_agency  204637 non-null  object 
 11  nif_contracting_agency   204637 non-null  int64  
 12  publication_date         204637 non-null  object 
 13  close_date               71851 non-null   object 
 14  cpvs

At first glance we can see that there are some columns with a lot of null values. The data types vary between object, float64 and int64. The data frames corresponding to 2018 to 2022 have 27 columns and those corresponding to 2023, 2024 and 2025 have 35, with different column names.
The dataframe corresponding to the PRR has no missing values, the data types vary between object, int64 and float64 and the column names are also different.
The missing values will be examined later. As for the columns, we'll analyse which ones are necessary for the analysis, remove those that aren't and then standardise the names of those that are.

#### **Inspecting null values**


In [229]:
for name, df in datasets.items():
    null_counts = df.isnull().sum()
    null_percent = (null_counts / len(df)) * 100
    null_report = pd.concat([null_counts, null_percent], axis=1)
    null_report.columns = ['null_count', 'null_percent']
    null_report = null_report[null_report['null_count'] > 0].sort_values(by='null_count', ascending=False)

    print(f"{name} — Missing Values Report:\n")
    print(null_report.round(2), "\n")


df_2018 — Missing Values Report:

                        null_count  null_percent
invitees                    204593         99.98
causes_deadline_change      191831         93.74
causes_price_change         176684         86.34
close_date                  132786         64.89
final_price                 132145         64.58
name_contestants             85921         41.99
document_id                  63679         31.12
document_name                63679         31.12
centralized_procedure            6          0.00
name_contracted                  1          0.00 

df_2019 — Missing Values Report:

                        null_count  null_percent
invitees                    286841         99.99
causes_deadline_change      264599         92.24
causes_price_change         247269         86.20
final_price                 192803         67.21
close_date                  191018         66.59
document_id                 103254         35.99
document_name               103254         35.99

Columns with the highest percentage of missing values are columns not required for the analyses to be performed. This will be drop later on.

### Data exploration conclusions

Before preprocessing, we identified key issues in the raw contract datasets:

- **Inconsistent schemas across years:** Column names and formats vary significantly between 2018–2022 (CSV) and 2023–2025 (JSON), requiring careful standardization.
- **Non-uniform ID columns:** Contract identifiers appear under different names (`contract_id`, `idcontrato`, `cd_base_gov`) and had to be unified to enable merging.
- **Many irrelevant columns:** Several fields (e.g., `link`, `observacoes`, `concorrentes`) are not useful for our analysis and were dropped.
- **Missing or incomplete values:** Key fields like `final_price` or `cpv` are often missing or inconsistently filled.
- **Separate PRR data:** PRR contracts were provided in a separate file, requiring a join based on `contract_id`. A new column `is_prr` was created to flag these cases.

These issues reinforced the need for a robust preprocessing pipeline to clean, unify, and enrich the data before analysis.


####  **Standardized Column Mapping Across Datasets (2018–2025 + PRR)**

|  Concept                  |  2018–2022               |  2023–2025                |  Keep? |  Standardized Name         |
|----------------------------|----------------------------|-----------------------------|---------|------------------------------|
| Contract ID                | `contract_id`              | `idcontrato`                | ✔️      | `contract_id`                |
| Contract type              | `contract_type`            | `tipocontrato`              | ✔️      | `contract_type`              |
| Procedure type             | `procedure_type`           | `tipoprocedimento`          | ✔️      | `procedure_type`             |
| Contracting entity         | `name_contracting_agency`  | `adjudicante`               | ✔️      | `contracting_entity`         |
| Contracted entity          | `name_contracted`          | `adjudicatarios`            | ✔️      | `contracted_entity`          |
| Contracted entity NIF      | `nif_contracted`           | —                           | ✔️      | `nif_contracted`             |
| Publication date           | `publication_date`         | `datapublicacao`            | ✔️      | `publication_date`           |
| CPV code                   | `cpvs`                     | `cpv`                       | ✔️      | `cpv`                        |
| Initial price              | `initial_price`            | `precobaseprocedimento`     | ✔️      | `initial_price`              |
| Contracted price           | —                          | `precocontratual`           | ✔️      | `contract_value`             |
| Final/Effective price      | `final_price`              | `precototalefetivo`         | ✔️      | `final_price`                |
| Contract year              | `contract_year`            | `ano`                       | ✔️      | `contract_year`              |
| Execution location         | `execution_location`       | `localexecucao`             | ✔️      | `execution_location`         |
| Document name              | `document_name`            | `desccontrato`              | ❌      |                              |
| Contract summary           | —                          | `objectocontrato`           | ❌      |                              |
| Unnamed index              | `unnamed:0`                | —                           | ❌      |                              |
| Contestants                | `name_contestants`         | —                           | ❌      |                              |
| Contestant NIFs            | `nif_contestants`          | —                           | ❌      |                              |
| Invitees                   | `invitees`                 | —                           | ❌      |                              |
| Centralized procedure      | `centralized_procedure`    | `procedimentocentralizado`  | ❌      |                              |
| Close date                 | `close_date`               | `datafechocontrato`         | ❌      |                              |
| Execution deadline         | `execution_deadline`       | `prazoexecucao`             | ❌      |                              |
| Signing date               | `signing_date`             | `datacelebracaocontrato`    | ❌      |                              |
| Environmental criteria     | `environmental_criteria`   | `contratoecologico`         | ❌      |                              |
| Material criteria          | `material_criteria`        | `critmateriais`             | ❌      |                              |
| Cause of deadline change   | `causes_deadline_change`   | —                           | ❌      |                              |
| Cause of price change      | `causes_price_change`      | —                           | ❌      |                              |
| Justification              | —                          | `fundamentacao`             | ❌      |                              |
| Framework agreement        | —                          | `numacordoquadro`           | ❌      |                              |
| Decision date              | —                          | `datadecisaoadjudicacao`    | ❌      |                              |
| Legal regime               | —                          | `regime`                    | ❌      |                              |
| Contract end type          | —                          | `tipofimcontrato`           | ❌      |                              |
| Number of competitors      | —                          | `concorrentes`              | ❌      |                              |
| Link to documents          | —                          | `link`                      | ❌      |                              |
| Observations               | —                          | `observacoes`               | ❌      |                              |
| Adjustment justification   | —                          | `fundajustedireto`          | ❌      |                              |
| Belongs to PRR (to be created) | —                     | —                           | ✔️      | `is_prr`                     |
| Macro CPV (to be created) | —                     | —                           | ✔️      | `cpv_macro`                     |
| Macro CPV description (to be created) | —                     | —                           | ✔️      | `cpv_macro_desc`                     |

## Preprocessing

#### **Set contract_id as index**


In [235]:
#List possible names of the contract Id column in the dataframes
possible_id_cols = ['contract_id', 'idcontrato', 'cd_base_gov']

def set_contract_id_as_index(df, df_name):
    #Identify the possible column name that are in the dataframe
    found_col = next((col for col in possible_id_cols if col in df.columns), None)

    if found_col:
        #Rename the identified column to 'contract_id' and set it as the index
        df.rename(columns={found_col: 'contract_id'}, inplace=True)
        df.set_index('contract_id', inplace=True)
        #To track if the alterations were made
        print(f"{df_name}: '{found_col}' renamed and set as 'contract_id' index.")
    else:
        print(f"{df_name}: No known ID column found.")

#Applying the function to all dataframes
for name, df in datasets.items():
    set_contract_id_as_index(df, name)

df_2018: 'contract_id' renamed and set as 'contract_id' index.
df_2019: 'contract_id' renamed and set as 'contract_id' index.
df_2020: 'contract_id' renamed and set as 'contract_id' index.
df_2021: 'contract_id' renamed and set as 'contract_id' index.
df_2022: 'contract_id' renamed and set as 'contract_id' index.
df_cpv: No known ID column found.
df_2023: 'idcontrato' renamed and set as 'contract_id' index.
df_2024: 'idcontrato' renamed and set as 'contract_id' index.
df_2025: 'idcontrato' renamed and set as 'contract_id' index.
df_prr: 'cd_base_gov' renamed and set as 'contract_id' index.


In [236]:
#Check if there are duplicates in 'contract_id' after indexing
for name, df in datasets.items():
    if df.index.name == 'contract_id':
        is_unique = df.index.is_unique
        print(f"{name}: {'Unique index' if is_unique else 'Duplicate contract_id(s) found'}")

df_2018: Duplicate contract_id(s) found
df_2019: Duplicate contract_id(s) found
df_2020: Duplicate contract_id(s) found
df_2021: Duplicate contract_id(s) found
df_2022: Duplicate contract_id(s) found
df_2023: Duplicate contract_id(s) found
df_2024: Duplicate contract_id(s) found
df_2025: Duplicate contract_id(s) found
df_prr: Duplicate contract_id(s) found


This is issue will be addressed later on.

#### **Standardize strings**

The dataframes have columns with different names that will the standardize by applying the **standardize_column** function (created below). 
Some columns have lists in the rows, extra spaces and other issues, this will be standardize and cleaned by applying the **clean_string_columns** function (created below).

In [239]:
# Standardize column names across dataframes: lowercase, no spaces or special characters
def standardize_columns(df):
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("-", "_")
        .str.replace(r"[^\w\s]", "", regex=True)
    )
    return df

#Clean string values that come as lists or stringified lists
def clean_string_columns(df, string_columns):
    for col in string_columns:
        if col in df.columns:
            def clean_value(x):
                val = x

                #Real Python list with one item
                if isinstance(x, list) and len(x) == 1:
                    val = x[0]

                #String that looks like a list ("['value']")
                elif isinstance(x, str) and x.startswith("[") and x.endswith("]"):
                    try:
                        parsed = ast.literal_eval(x)
                        if isinstance(parsed, list) and parsed:
                            val = parsed[0]
                        else:
                            return None
                    except:
                        # fallback cleanup if parsing fails (malformed string)
                        val = x.strip("[]").strip()

                #Remove stray quotes and extra whitespace
                if isinstance(val, str):
                    return val.strip(" '\"")

                return val

            #Apply cleaning safely with .loc to avoid warnings
            df.loc[:, col] = df[col].apply(clean_value)

    return df

# Apply standardization and cleaning to all datasets
for name, df in datasets.items():
    df = standardize_columns(df)
    df = clean_string_columns(df, ['contract_type', 'procedure_type', 'contracting_entity'])
    datasets[name] = df  #Save cleaned dataframe back into the dictionary
    print(f"{name}:Columns and string values standardized.")

df_2018:Columns and string values standardized.
df_2019:Columns and string values standardized.
df_2020:Columns and string values standardized.
df_2021:Columns and string values standardized.
df_2022:Columns and string values standardized.
df_cpv:Columns and string values standardized.
df_2023:Columns and string values standardized.
df_2024:Columns and string values standardized.
df_2025:Columns and string values standardized.
df_prr:Columns and string values standardized.


The contract_type columns has rows with more than one contract type in it. This will be split below.

In [241]:
#Expand contract_type into multiple columns (before filtering and concat)

def expand_contract_type_column(df, column='contract_type'):
    def to_list(x):
        if isinstance(x, list):
            return x
        if isinstance(x, str) and x.startswith("[") and x.endswith("]"):
            try:
                parsed = ast.literal_eval(x)
                if isinstance(parsed, list):
                    return parsed
            except:
                return [x.strip("[]").strip(" '\"")]
        return [x] if pd.notnull(x) else []

    expanded = df[column].apply(to_list)
    max_items = max(1, expanded.map(len).max())  # sempre no mínimo 1 coluna

    for i in range(max_items):
        df.loc[:, f"{column}_{i+1}"] = expanded.apply(lambda x: x[i] if i < len(x) else None)

    return df


#### **Rename columns names**

As we have seen, the columns in the different dataframes have different names for the same attribute. The columns used in the analysis are renamed according to the table already presented.

In [243]:
#Creating renaming dictionaries
rename_2018_2022 = {
    "contract_id": "contract_id",
    "contract_type": "contract_type",
    "procedure_type": "procedure_type",
    "name_contracting_agency": "contracting_entity",
    "name_contracted": "contracted_entity",
    "nif_contracted": "nif_contracted",
    "publication_date": "publication_date",
    "cpvs": "cpv",
    "initial_price": "initial_price",
    "final_price": "final_price",
    "contract_year": "contract_year",
    "execution_location": "execution_location"
}

rename_2023_2025 = {
    "idcontrato": "contract_id",
    "tipocontrato": "contract_type",
    "tipoprocedimento": "procedure_type",
    "adjudicante": "contracting_entity",
    "adjudicatarios": "contracted_entity",
    "datapublicacao": "publication_date",
    "cpv": "cpv",
    "precobaseprocedimento": "initial_price",
    "precocontratual": "contract_value",
    "precototalefetivo": "final_price",
    "ano": "contract_year",
    "localexecucao": "execution_location"
}

#Apply renaming to dataframe from 2018–2022
for year in range(2018, 2023):
    df = datasets[f'df_{year}']
    df.rename(columns=rename_2018_2022, inplace=True)
    print(f"df_{year}: Columns renamed")

#Apply renaming to dataframes from 2023–2025
for year in range(2023, 2026):
    df = datasets[f'df_{year}']
    df.rename(columns=rename_2023_2025, inplace=True)
    print(f"df_{year}: Columns renamed")

#Apply expansion to all dataframes
for year in range(2018, 2026):
    key = f"df_{year}"
    if 'contract_type' in datasets[key].columns:
        datasets[key] = expand_contract_type_column(datasets[key], 'contract_type')
        print(f"{key}: Expanded contract_type into multiple columns")


#Rename df_cpv (macro CPV) to standardized names
datasets['df_cpv'] = datasets['df_cpv'].rename(columns={
    'código_cpv_2_dígitos': 'cpv_macro',
    'descrição': 'cpv_macro_desc'
})

# Ensure cpv_macro is string and zero-padded
datasets['df_cpv']['cpv_macro'] = datasets['df_cpv']['cpv_macro'].astype(str).str.zfill(2)
print("df_cpv: Columns renamed")


df_2018: Columns renamed
df_2019: Columns renamed
df_2020: Columns renamed
df_2021: Columns renamed
df_2022: Columns renamed
df_2023: Columns renamed
df_2024: Columns renamed
df_2025: Columns renamed
df_2018: Expanded contract_type into multiple columns
df_2019: Expanded contract_type into multiple columns
df_2020: Expanded contract_type into multiple columns
df_2021: Expanded contract_type into multiple columns
df_2022: Expanded contract_type into multiple columns
df_2023: Expanded contract_type into multiple columns
df_2024: Expanded contract_type into multiple columns
df_2025: Expanded contract_type into multiple columns
df_cpv: Columns renamed


#### **Creating boolean flag for PRR**

To keep track of the contracts that are inserted in the PRR, we will create a column **'is_prr'** with a boolean flag, so that when the dataframes are merged, we will be able to distinguish the contracts.

In [245]:
#Add boolean flag to identify PRR contracts
datasets["df_prr"].loc[:, "is_prr"] = True

#### **Removing columns we will not use**


In [247]:
#List of standardized column names to keep
columns_to_keep = [
    "contract_id",
    "contract_type",
    "procedure_type",
    "contracting_entity",
    "contracted_entity",
    "nif_contracted",
    "publication_date",
    "cpv",
    "initial_price",
    "contract_value",
    "final_price",
    "contract_year",
    "execution_location",
    "is_prr", # will be created later
    "cd_projeto",
    "cpv_macro",
    "cpv_macro_desc"
]

#Keep only selected columns, including dynamically detected contract_type_n columns
def filter_columns(df):
    dynamic_contract_type_cols = [col for col in df.columns if col.startswith("contract_type_")]
    all_keep = columns_to_keep + dynamic_contract_type_cols
    return df[[col for col in df.columns if col in all_keep]]

#### **Merging datasets**


In [249]:
#Concatenate all dataframes for public contracts from 2018 to 2025
df_contracts_all = pd.concat(
    [datasets[f"df_{year}"] for year in range(2018, 2026)],
    axis=0
)

#### **Creating CPV and PRR**

After concatenate all the dataframes regarding the public contracts from 2018 to 2025, we will merge the **CPV** and **PRR** dataframes. However, first will need to do some changes.

In [251]:
## Feature engineering for CPV

#For this 'contract_id' must be a regular column (not index)
df_contracts_all = df_contracts_all.reset_index() if 'contract_id' not in df_contracts_all.columns else df_contracts_all

#Extract the first CPV code
df_contracts_all['cpv_code'] = df_contracts_all['cpv'].astype(str).str.extract(r'(\d{8}-\d)')

#Extract only the first CPV description cleaned
df_contracts_all['cpv_desc'] = df_contracts_all['cpv'].astype(str)\
    .str.extract(r'\d{8}-\d\s*-\s*([^,\]]+)')\
    .apply(lambda x: x.strip(" '\"") if isinstance(x, str) else x)


#Extract CPV macro (first 2 digits of cpv_code)
df_contracts_all['cpv_macro'] = df_contracts_all['cpv_code'].str[:2]

#Merge CPV macro descriptions
df_contracts_all = df_contracts_all.merge(
    datasets['df_cpv'], on='cpv_macro', how='left'
)

#Create a mapping from cpv_code to cpv_desc using 2023–2025 (higher quality descriptions)
cpv_code_desc = df_contracts_all[
    df_contracts_all['contract_year'].between(2023, 2025)
][['cpv_code', 'cpv_desc']].dropna().drop_duplicates()

#Merge cpv_code → cpv_desc mapping into full dataset as cpv_desc_final
df_contracts_all = df_contracts_all.merge(
    cpv_code_desc.rename(columns={'cpv_desc': 'cpv_desc_final'}),
    on='cpv_code', how='left'
)

#Fill in missing cpv_desc_final values with raw extraction (from earlier)
df_contracts_all['cpv_desc_final'] = df_contracts_all['cpv_desc_final'].combine_first(df_contracts_all['cpv_desc'])

#Clean trailing quotes or spaces from cpv_desc_final
df_contracts_all['cpv_desc_final'] = df_contracts_all['cpv_desc_final'].str.strip(" '\"")

##Drop intermediate cpv_desc column
df_contracts_all.drop(columns=['cpv_desc'], inplace=True)

In [252]:
##Feature engineering for PRR

#Prepare df_prr and mark contracts as PRR-related
df_prr = datasets["df_prr"]
df_prr = df_prr.reset_index() if 'contract_id' not in df_prr.columns else df_prr
df_prr_ids = (
    df_prr
      .reset_index()
      [['contract_id','cd_projeto']]
      .drop_duplicates()
      .assign(is_prr=True)
)

#Merge PRR flags into full dataset
df_merged = (
    df_contracts_all
      .merge(df_prr_ids, on='contract_id', how='left')
      .assign(
          is_prr=lambda d: d['is_prr'].fillna(False).astype('boolean')
      )
)

#Apply final column filtering after merge
df_merged = filter_columns(df_merged)

#Reorder contract_type_* columns to appear at the end
contract_type_cols = sorted([col for col in df_merged.columns if col.startswith("contract_type_")])
other_cols = [col for col in df_merged.columns if col not in contract_type_cols]
df_merged = df_merged[other_cols + contract_type_cols]

#Set contract_id as index again
df_merged.set_index('contract_id', inplace=True)

#Final check for duplicates
duplicate_count = df_merged.index.duplicated().sum()
print(f"df_merged created — Duplicated contract_id entries: {duplicate_count}")


C:\Users\carol\AppData\Local\Temp\ipykernel_30072\3068516149.py:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_prr=lambda d: d['is_prr'].fillna(False).astype('boolean')


df_merged created — Duplicated contract_id entries: 1038626


In [253]:
df_merged.shape

(2212199, 19)

#### **Removing duplicates**

After the merged dataframe was created, we can see that are 1038626 duplicated contract_id entries, this will be address next.

In [255]:
#Count how many contract_id values are duplicated
duplicate_counts = df_merged.index.value_counts()
duplicated_ids = duplicate_counts[duplicate_counts > 1]

print(f"Number of contract_id with more than one entry: {duplicated_ids.shape[0]}")

#Display the duplicated contract_id entries sorted by contract_id
duplicated_rows = df_merged.loc[duplicated_ids.index].sort_index()

#Show a preview of duplicated rows
print("\nPreview of duplicated contract_id entries:")
display(duplicated_rows.head(50))  # show only first 10 for safety


Number of contract_id with more than one entry: 177440

Preview of duplicated contract_id entries:


,contracted_entity,nif_contracted,contract_type,procedure_type,contracting_entity,publication_date,cpv,execution_location,initial_price,final_price,contract_year,contract_value,cpv_macro,cpv_macro_desc,cd_projeto,is_prr,contract_type_1,contract_type_2,contract_type_3
contract_id,,,,,,,,,,,,,,,,,,,
2552471,"Espalha Ideias - Actividades de Tempos Livres,...",505323184,Aquisição de serviços,Concurso público,Universidade de Lisboa,2016-10-20,92600000-7,"[{'country': 'Portugal', 'district': 'Lisboa',...",1420297.51,NaN,2019,NaN,92,"Serviços recreativos, culturais e desportivos",NaN,False,Aquisição de serviços,NaN,NaN
2552471,"Espalha Ideias - Actividades de Tempos Livres,...",505323184,Aquisição de serviços,Concurso público,Universidade de Lisboa,2016-10-20,92600000-7,"[{'country': 'Portugal', 'district': 'Lisboa',...",1420297.51,NaN,2019,NaN,92,"Serviços recreativos, culturais e desportivos",NaN,False,Aquisição de serviços,NaN,NaN
2552471,"Espalha Ideias - Actividades de Tempos Livres,...",505323184,Aquisição de serviços,Concurso público,Universidade de Lisboa,2016-10-20,92600000-7,"[{'country': 'Portugal', 'district': 'Lisboa',...",1420297.51,NaN,2019,NaN,92,"Serviços recreativos, culturais e desportivos",NaN,False,Aquisição de serviços,NaN,NaN
2552471,"Espalha Ideias - Actividades de Tempos Livres,...",505323184,Aquisição de serviços,Concurso público,Universidade de Lisboa,2016-10-20,92600000-7,"[{'country': 'Portugal', 'district': 'Lisboa',...",1420297.51,NaN,2019,NaN,92,"Serviços recreativos, culturais e desportivos",NaN,False,Aquisição de serviços,NaN,NaN
2552471,"Espalha Ideias - Actividades de Tempos Livres,...",505323184,Aquisição de serviços,Concurso público,Universidade de Lisboa,2016-10-20,92600000-7,"[{'country': 'Portugal', 'district': 'Lisboa',...",1420297.51,NaN,2019,NaN,92,"Serviços recreativos, culturais e desportivos",NaN,False,Aquisição de serviços,NaN,NaN
2552471,"Espalha Ideias - Actividades de Tempos Livres,...",505323184,Aquisição de serviços,Concurso público,Universidade de Lisboa,2016-10-20,92600000-7,"[{'country': 'Portugal', 'district': 'Lisboa',...",1420297.51,NaN,2019,NaN,92,"Serviços recreativos, culturais e desportivos",NaN,False,Aquisição de serviços,NaN,NaN
2552471,"Espalha Ideias - Actividades de Tempos Livres,...",505323184,Aquisição de serviços,Concurso público,Universidade de Lisboa,2016-10-20,92600000-7,"[{'country': 'Portugal', 'district': 'Lisboa',...",1420297.51,NaN,2019,NaN,92,"Serviços recreativos, culturais e desportivos",NaN,False,Aquisição de serviços,NaN,NaN
2552471,"Espalha Ideias - Actividades de Tempos Livres,...",505323184,Aquisição de serviços,Concurso público,Universidade de Lisboa,2016-10-20,92600000-7,"[{'country': 'Portugal', 'district': 'Lisboa',...",1420297.51,NaN,2019,NaN,92,"Serviços recreativos, culturais e desportivos",NaN,False,Aquisição de serviços,NaN,NaN
2552471,"Espalha Ideias - Actividades de Tempos Livres,...",505323184,Aquisição de serviços,Concurso público,Universidade de Lisboa,2016-10-20,92600000-7,"[{'country': 'Portugal', 'district': 'Lisboa',...",1420297.51,NaN,2019,NaN,92,"Serviços recreativos, culturais e desportivos",NaN,False,Aquisição de serviços,NaN,NaN


After analyzing, the duplicates signaled really are duplicates that are going to be removed.

In [257]:
#Remove duplicated contract_id entries, keeping only the first occurrence
df_merged = df_merged[~df_merged.index.duplicated(keep='first')]

#Confirming the result
print("Remaining duplicated contract_id entries:", df_merged.index.duplicated().sum())


Remaining duplicated contract_id entries: 0


#### Preprocessing columns to support data analysis

##### Converting dates to datetime format

Clean the columns with dates to analysis.

In [260]:
df_merged = df_merged.copy()

#Custom function to extract and parse potential date strings
def parse_publication_date(value):
    if pd.isna(value):
        return None

    #Extract potential date string in known formats
    match = re.search(r'\d{4}-\d{2}-\d{2}|\d{2}/\d{2}/\d{4}', str(value))
    if not match:
        return None

    date_str = match.group()

    #Try parsing
    for fmt in ("%Y-%m-%d", "%d/%m/%Y"):
        try:
            return datetime.strptime(date_str, fmt)
        except:
            continue
    return None

#Apply parsing logic
df_merged['publication_date'] = df_merged['publication_date'].apply(parse_publication_date)

#Ensure column is in pandas datetime format
df_merged['publication_date'] = pd.to_datetime(df_merged['publication_date'], errors='coerce')

#Extract year
df_merged['year'] = df_merged['publication_date'].dt.year

#Final check
valid_dates = df_merged['publication_date'].notna().sum()
print(f"Parsed dates: {valid_dates} valid / {len(df_merged)} total rows")


Parsed dates: 1173573 valid / 1173573 total rows


##### Initial price vs Final price vs Contract year

For some contracts there are two different values, the **initial_price** and the **final_price**, in some contracts this values are equal for others not.

In [262]:
#Understanding values of contracts, which we have 4 columns, and some of them with missing values.
df_merged.describe()

,publication_date,initial_price,final_price,contract_year,contract_value,year
count,1173573,1.173573e+06,6.615790e+05,1.173573e+06,4.828470e+05,1.173573e+06
mean,2022-03-31 00:04:40.497251072,6.064157e+05,1.900172e+04,2.021634e+03,8.303812e+04,2.021767e+03
min,2016-10-20 00:00:00,-3.120000e+04,0.000000e+00,2.018000e+03,-1.056600e+07,2.016000e+03
25%,2020-07-24 00:00:00,4.422600e+03,0.000000e+00,2.020000e+03,3.586075e+03,2.020000e+03
50%,2022-05-19 00:00:00,1.396500e+04,0.000000e+00,2.022000e+03,1.160461e+04,2.022000e+03
75%,2024-02-06 00:00:00,4.650000e+04,6.000000e+03,2.023000e+03,3.287190e+04,2.024000e+03
max,2025-05-17 00:00:00,3.412802e+10,5.078584e+08,2.025000e+03,3.795000e+08,2.025000e+03
std,NaN,3.244789e+07,8.180242e+05,2.122251e+00,1.264136e+06,2.094366e+00


In the **initial_price** and **contract_value** there are negative values. This question will be examined.

In [264]:
df_merged[df_merged['initial_price'] < 0]


,contracted_entity,nif_contracted,contract_type,procedure_type,contracting_entity,publication_date,cpv,execution_location,initial_price,final_price,contract_year,contract_value,cpv_macro,cpv_macro_desc,cd_projeto,is_prr,contract_type_1,contract_type_2,contract_type_3,year
contract_id,,,,,,,,,,,,,,,,,,,,
9708572,"[513948848 - GJMF - INVESTIMENTOS, UNIPESSOAL,...",NaN,[Aquisição de serviços],Ajuste Direto Regime Geral,[506613399 - Município de Góis],2023-01-17,"[71000000-8 - Serviços de arquitectura, constr...","[Portugal, Coimbra, Góis]",-789.8,0.0,2023,-789.8,71,Serviços de arquitetura e engenharia,NaN,False,Aquisição de serviços,None,None,2023
9922408,"[516884581 - DOCE100ABELHA UNIPESSOAL, LDA]",NaN,[Aquisição de serviços],Ajuste Direto Regime Geral,[507880803 - Escola Superior de Enfermagem do ...,2023-04-11,[55330000-2 - Serviços de cafetaria],"[Portugal, Porto, Porto]",-4950.0,0.0,2023,4950.0,55,Serviços de hotelaria e restauração,NaN,False,Aquisição de serviços,None,None,2023
9938909,[501426230 - ICA - INDUSTRIA E COMERCIO ALIMEN...,NaN,[Concessão de serviços públicos],Ajuste Direto Regime Geral,[510089224 - Instituto Português do Desporto e...,2023-04-19,[55330000-2 - Serviços de cafetaria],"[Portugal, Lisboa, Lisboa]",-31200.0,0.0,2023,31200.0,55,Serviços de hotelaria e restauração,NaN,False,Concessão de serviços públicos,None,None,2023
11220782,[- - Maria Gorete Santos Silva],NaN,[Concessão de serviços públicos],Ajuste Direto Regime Geral,[600008878 - Guarda Nacional Republicana],2025-02-12,"[55000000-0 - Serviços de hotelaria, restauraç...","[Portugal, Aveiro, Aveiro]",-5400.0,0.0,2024,6624.0,55,Serviços de hotelaria e restauração,NaN,False,Concessão de serviços públicos,None,None,2025
11217607,"[505009161 - BEIRA BIRA, LDA]",NaN,[Concessão de serviços públicos],Consulta Prévia,[600008878 - Guarda Nacional Republicana],2025-02-10,"[55000000-0 - Serviços de hotelaria, restauraç...","[Portugal, Aveiro]",-2250.0,0.0,2025,2418.0,55,Serviços de hotelaria e restauração,NaN,False,Concessão de serviços públicos,None,None,2025


After investigating these figures, they really are negative, since the amount should be returned to the state.

In [266]:
df_merged[df_merged['contract_value'] < 0]

,contracted_entity,nif_contracted,contract_type,procedure_type,contracting_entity,publication_date,cpv,execution_location,initial_price,final_price,contract_year,contract_value,cpv_macro,cpv_macro_desc,cd_projeto,is_prr,contract_type_1,contract_type_2,contract_type_3,year
contract_id,,,,,,,,,,,,,,,,,,,,
9708572,"[513948848 - GJMF - INVESTIMENTOS, UNIPESSOAL,...",NaN,[Aquisição de serviços],Ajuste Direto Regime Geral,[506613399 - Município de Góis],2023-01-17,"[71000000-8 - Serviços de arquitectura, constr...","[Portugal, Coimbra, Góis]",-789.80,0.0,2023,-789.80,71,Serviços de arquitetura e engenharia,NaN,False,Aquisição de serviços,None,None,2023
10215697,[500364460 - JCDECAUX PORTUGAL - MOBILIÁRIO U...,NaN,[Concessão de serviços públicos],Concurso público,[500051062 - Município de Sintra],2023-08-30,[71410000-5 - Serviços de planeamento urbano],"[Portugal, Lisboa, Sintra]",0.00,0.0,2023,-9774000.00,71,Serviços de arquitetura e engenharia,NaN,False,Concessão de serviços públicos,None,None,2023
10215705,[502141549 - CEMARK – MOBILIÁRIO URBANO E PUBL...,NaN,[Concessão de serviços públicos],Concurso público,[500051062 - Município de Sintra],2023-08-30,[71410000-5 - Serviços de planeamento urbano],"[Portugal, Lisboa, Sintra]",0.00,0.0,2023,-10566000.00,71,Serviços de arquitetura e engenharia,NaN,False,Concessão de serviços públicos,None,None,2023
10215707,[502141549 - CEMARK – MOBILIÁRIO URBANO E PUBL...,NaN,[Concessão de serviços públicos],Concurso público,[500051062 - Município de Sintra],2023-08-30,[71410000-5 - Serviços de planeamento urbano],"[Portugal, Lisboa, Sintra]",0.00,0.0,2023,-9630000.00,71,Serviços de arquitetura e engenharia,NaN,False,Concessão de serviços públicos,None,None,2023
10215722,[502141549 - CEMARK – MOBILIÁRIO URBANO E PUBL...,NaN,[Concessão de serviços públicos],Concurso público,[500051062 - Município de Sintra],2023-08-30,[71410000-5 - Serviços de planeamento urbano],"[Portugal, Lisboa, Sintra]",0.00,0.0,2023,-2292000.00,71,Serviços de arquitetura e engenharia,NaN,False,Concessão de serviços públicos,None,None,2023
10486390,"[504197088 - PODIUM EVENTS, SA]",NaN,[Concessão de serviços públicos],Concurso público,[500051062 - Município de Sintra],2024-01-19,[92622000-7 - Serviços de organização de manif...,"[Portugal, Lisboa, Sintra]",0.00,0.0,2024,-207000.00,92,"Serviços recreativos, culturais e desportivos",NaN,False,Concessão de serviços públicos,None,None,2024
10543799,[503096024 - A SUPER 2000 - MAQUINAS AUTOMATIC...,NaN,[Concessão de serviços públicos],Ajuste Direto Regime Geral,[600008878 - Guarda Nacional Republicana],2024-02-06,[51000000-9 - Serviços de instalação (excepto ...,"[Portugal, Lisboa, Lisboa]",0.00,0.0,2024,-7853.66,51,Serviços de instalação (exceto software),NaN,False,Concessão de serviços públicos,None,None,2024
10597943,"[507778553 - H Sarah Trading, Unipessoal, Lda.]",NaN,[Outros],Ajuste Direto Regime Geral,[501280740 - Município de Ourém],2024-03-06,"[98000000-3 - Outros serviços comunitários, so...","[Portugal, Santarém, Ourém]",0.00,0.0,2024,-17820.00,98,"Outros serviços comunitários, sociais e pessoais",NaN,False,Outros,None,None,2024
10795406,[- - Bruno Manuel Ferreira Neto],NaN,[Aquisição de serviços],Ajuste Direto Regime Geral,"[510838260 - União das Freguesias de Nogueira,...",2024-06-28,[79714000-2 - Serviços de vigilância],"[Portugal, Braga, Braga]",2997.00,2637.0,2024,-2997.00,79,Serviços educacionais e formação,NaN,False,Aquisição de serviços,None,None,2024


After analysis, this values appears at the contract as value to be received by the State.
All the values are correct, thus will be mantained.

In order to analyze the values of the contracts, it was decided to the contracts that have a **final_price** or a different value from the **initial_price** the higher value is mantained.

In [268]:
#Select the best available price based on contract year

def select_best_price(row):
    if row['contract_year'] <= 2022:
        return row['initial_price'] if pd.notna(row['initial_price']) else row['contract_value']
    else:
        return row['contract_value'] if pd.notna(row['contract_value']) else row['initial_price']

df_merged.loc[:, 'contract_price_analysis'] = df_merged.apply(select_best_price, axis=1)


##### Retrieve the nif's from contracted_entity and standardize the names of the entities

After exploration, it was found that the column **contracted_entity** has the nifs before the name of the entity, which is missing in **nif_contracted**. Also, the names of the entities are written in different formats, based on the nif we will standardize the names for the one that is more commonly used.

In [270]:
df_merged = df_merged.copy()

#Ensure 'contracted_entity' is a string
df_merged['contracted_entity'] = df_merged['contracted_entity'].astype(str)

#Remove brackets or quotes, it was observed in some entities name
df_merged['contracted_entity'] = df_merged['contracted_entity'].str.replace(
    r"^\[?['\"]?|['\"]?\]?$", '', regex=True)

#Find the rows with missing nifs
missing_nif_rows = df_merged['nif_contracted'].isnull()

#Extract nif from 'contracted_entity' when exists
nif_pattern = r'^([A-Z]{0,3}-?\d{6,15}|\d{2}[-\s]\d{6,10})[\s\-–—]+'
extracted_nifs = df_merged.loc[missing_nif_rows, 'contracted_entity'].str.extract(nif_pattern, expand=False)

#Fill in the extracted nifs
df_merged.loc[missing_nif_rows, 'nif_contracted'] = extracted_nifs

#Remove nifs from 'contracted_entity' to mantained just the name
df_merged['contracted_entity'] = df_merged['contracted_entity'].str.replace(nif_pattern, '', regex=True)
df_merged['contracted_entity'] = df_merged['contracted_entity'].str.replace(r'[A-Z]{0,3}-?\d{6,15}', '', regex=True)

#Split rows with multiple entities into lists, it was observed in some entities name
df_merged['contracted_entity'] = df_merged['contracted_entity'].str.split(r"',\s*'|\",\s*\"")

#Explode the list into separate rows
df_merged = df_merged.explode('contracted_entity').reset_index(drop=True)

#Clean up the names, like punctuation, spaces, HTML codes
df_merged['contracted_entity'] = df_merged['contracted_entity'].astype(str)
df_merged['contracted_entity'] = df_merged['contracted_entity'].str.replace('&amp;', '&', regex=False)
df_merged['contracted_entity'] = df_merged['contracted_entity'].str.replace(r'[-–—:,\.]+', ' ', regex=True)
df_merged['contracted_entity'] = df_merged['contracted_entity'].str.replace(r'\s+', ' ', regex=True)
df_merged['contracted_entity'] = df_merged['contracted_entity'].str.strip(" '\"")

#Build a dictionary for each nif to find the most commonly used name
valid_nifs = df_merged[df_merged['nif_contracted'].notnull()]
name_counts = valid_nifs.groupby(['nif_contracted', 'contracted_entity']).size().reset_index(name='count')
name_counts = name_counts.sort_values(['nif_contracted', 'count'], ascending=[True, False])
name_mapping = name_counts.drop_duplicates('nif_contracted').set_index('nif_contracted')['contracted_entity'].to_dict()

#Apply the mapping to create a clean name column
def choose_clean_name(row):
    if row['nif_contracted'] in name_mapping:
        return name_mapping[row['nif_contracted']]
    else:
        return row['contracted_entity']

df_merged['clean_contracted_entity'] = df_merged.apply(choose_clean_name, axis=1)

#Check the results
print("Nifs extracted:", extracted_nifs.notnull().sum())
print("Still missing nifs:", df_merged['nif_contracted'].isnull().sum())
print(df_merged[['contracted_entity', 'nif_contracted', 'clean_contracted_entity']].head(10))


Nifs extracted: 442444
Still missing nifs: 40502
                       contracted_entity nif_contracted  \
0                              Great Ink      513179763   
1                          Bricantel Lda      502888539   
2                  Contenur Portugal S A      502137770   
3     Speedmedia Serviços Multimédia Lda      504929518   
4         Transportes Silva Baptista Lda      501956174   
5        LABESFAL Laboratório Almiro S A      501169580   
6                  HTA HELICOPETEROS LDA      503811874   
7                    Mendes & Irmãos S A      500384436   
8  MADIGUIMA COMÉRCIO DE FRUTAS LIMITADA      503301205   
9               IBERDATA Equipamentos SA      501904921   

                             clean_contracted_entity  
0                                          Great Ink  
1  Bricantel Comércio de Material Electrico de Br...  
2                               Contenur Portugal SA  
3                 Speedmedia Serviços Multimédia Lda  
4                     Tran

In [271]:
df_merged[df_merged['nif_contracted'].isnull()]

,contracted_entity,nif_contracted,contract_type,procedure_type,contracting_entity,publication_date,cpv,execution_location,initial_price,final_price,...,cpv_macro,cpv_macro_desc,cd_projeto,is_prr,contract_type_1,contract_type_2,contract_type_3,year,contract_price_analysis,clean_contracted_entity
368715,Shanghai EBO Optoelectronic Technology CO Ltd,NaN,Aquisição de bens móveis,Ajuste Direto Regime Geral,"LIP, Laboratório de Instrumentação e Física Ex...",2021-01-26,38000000-5,"[{'country': 'Portugal', 'district': 'Coimbra'...",24058.88,23975.81,...,38,"Mobiliário, artigos de escritório, fornecimentos",NaN,False,Aquisição de bens móveis,NaN,NaN,2021,24058.88,Shanghai EBO Optoelectronic Technology CO Ltd
690731,José Teixeira Moreira,NaN,[Aquisição de serviços],Ajuste Direto Regime Geral,[510840744 - União das Freguesias de Unhão e L...,2023-01-02,[90611000-3 - Serviços de limpeza de ruas],"[Portugal, Porto, Felgueiras]",9120.00,7296.00,...,90,"Serviços de esgotos, resíduos, limpeza e ambiente",NaN,False,Aquisição de serviços,None,None,2023,9120.00,José Teixeira Moreira
690732,Maria Alice Martins Dias,NaN,[Aquisição de serviços],Consulta Prévia,"[510838626 - União das Freguesias de Pedreira,...",2023-01-02,[90600000-3 - Serviços de saneamento e limpeza...,"[Portugal, Porto, Felgueiras]",7704.00,7562.00,...,90,"Serviços de esgotos, resíduos, limpeza e ambiente",NaN,False,Aquisição de serviços,None,None,2023,7704.00,Maria Alice Martins Dias
690734,Carina da Conceição Marques Rodrigues Loureiro,NaN,[Aquisição de serviços],Ajuste Direto Regime Geral,[507853024 - Freguesia de Candoso (São Martinho)],2023-01-02,[85322000-2 - Programa de acção comunitária],"[Portugal, Braga, Guimarães]",11400.00,1140.00,...,85,NaN,NaN,False,Aquisição de serviços,None,None,2023,11400.00,Carina da Conceição Marques Rodrigues Loureiro
690737,Joana Filipa Ferreira dos Santos,NaN,[Aquisição de serviços],Ajuste Direto Regime Geral,[507853024 - Freguesia de Candoso (São Martinho)],2023-01-02,[75110000-0 - Serviços públicos gerais],"[Portugal, Braga, Guimarães]",9600.00,9600.00,...,75,Serviços administrativos e sociais,NaN,False,Aquisição de serviços,None,None,2023,9600.00,Joana Filipa Ferreira dos Santos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1182213,Joel Augusto Gonçalves Vieira,NaN,[Aquisição de serviços],Ajuste Direto Regime Geral,[501280740 - Município de Ourém],2025-05-16,[90911200-8 - Serviços de limpeza de edifícios],"[Portugal, Santarém, Ourém]",19900.00,0.00,...,90,"Serviços de esgotos, resíduos, limpeza e ambiente",NaN,False,Aquisição de serviços,None,None,2025,19850.00,Joel Augusto Gonçalves Vieira
1182215,Mário Fernandes Pedroso,NaN,[Aquisição de serviços],Concurso público,[500051062 - Município de Sintra],2025-05-16,[71410000-5 - Serviços de planeamento urbano],"[Portugal, Lisboa, Sintra]",106640.00,0.00,...,71,Serviços de arquitetura e engenharia,NaN,False,Aquisição de serviços,None,None,2025,106640.00,Mário Fernandes Pedroso
1182273,Álvaro Daniel da Costa Silva,NaN,[Aquisição de serviços],Ajuste Direto Regime Geral,[507044290 - Freguesia de Milheirós de Poiares],2025-05-16,"[92312200-3 - Serviços fornecidos por autores,...","[Portugal, Aveiro, Santa Maria da Feira]",31500.00,0.00,...,92,"Serviços recreativos, culturais e desportivos",NaN,False,Aquisição de serviços,None,None,2025,31500.00,Álvaro Daniel da Costa Silva
1182311,Antonio dos Santos Tavares,NaN,[Aquisição de serviços],Ajuste Direto Regime Geral,[507964446 - Freguesia de Ribeira de Fráguas],2025-05-16,[50230000-6 - Serviços de reparação e manutenç...,"[Portugal, Aveiro, Albergaria-a-Velha]",7000.00,0.00,...,50,Serviços de reparação e manutenção,NaN,False,Aquisição de serviços,None,None,2025,7000.00,Antonio dos Santos Tavares


After cleaning the 'contracted_entity' from the nif numbers, we still have null values in this column for individuals. After an exhaustive search to find these numbers on the base.gov platform, this remains missing.

##### Define the important events to be analysed

In [274]:
#Define key periods
covid_start = '2020-03-01'
covid_end   = '2021-12-31'
war_start   = '2022-02-24'
analysis_end = '2025-12-31'

In [275]:
#Create event period flags
df_merged.loc[:, 'event_period'] = 'pre_covid'
df_merged.loc[(df_merged['publication_date'] >= covid_start) & (df_merged['publication_date'] <= covid_end), 'event_period'] = 'covid'
df_merged.loc[(df_merged['publication_date'] >= war_start), 'event_period'] = 'ukraine_war'

In [276]:
df_merged.dtypes

contracted_entity                  object
nif_contracted                     object
contract_type                      object
procedure_type                     object
contracting_entity                 object
publication_date           datetime64[ns]
cpv                                object
execution_location                 object
initial_price                     float64
final_price                       float64
contract_year                       int64
contract_value                    float64
cpv_macro                          object
cpv_macro_desc                     object
cd_projeto                         object
is_prr                            boolean
contract_type_1                    object
contract_type_2                    object
contract_type_3                    object
year                                int32
contract_price_analysis           float64
clean_contracted_entity            object
event_period                       object
dtype: object

In [277]:
#Final df_merged to be exported
df_merged
df_merged.to_csv("df_merged.csv", index=True)
print("Saved to same folder as notebook")


Saved to same folder as notebook
